# 1. Import Dependencies

In [1]:
#Install SWIG https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering


In [2]:
# !pip install "pygame>=2.5.0"
# !pip install gym[box2d] 
!pip install pyglet==1.3.2


  Using cached pyglet-1.3.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
Using cached pyglet-1.3.2-py2.py3-none-any.whl (1.0 MB)
Using cached future-1.0.0-py3-none-any.whl (491 kB)

   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   ---------------------------------------- 0/2 [future]
   -----------------

In [18]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import os
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_


# 2. Test Environment

In [19]:
environment_name = "CarRacing-v2"


In [20]:
env = gym.make(environment_name, render_mode="human",)


In [9]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        # Convert action to native Python floats to avoid Box2D TypeError
        action = action.astype(float)
        n_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
# env.close()


Episode:1 Score:-32.20338983050896
Episode:2 Score:-31.972789115646762
Episode:3 Score:-40.29850746268721
Episode:4 Score:-26.470588235294457
Episode:5 Score:-38.053097345133416


In [10]:
env.close()


In [11]:
env.action_space.sample()


array([0.85058343, 0.17418566, 0.36640432], dtype=float32)

In [12]:
env.observation_space.sample()


array([[[140,  22, 213],
        [120, 100, 193],
        [189,   9, 247],
        ...,
        [186,  21, 119],
        [101, 167, 203],
        [ 28,  74,  32]],

       [[223, 248,  64],
        [ 75, 112,  70],
        [131, 214,  45],
        ...,
        [106, 188,  98],
        [212, 134, 102],
        [166, 233,  51]],

       [[ 47, 207, 138],
        [ 18,  24, 100],
        [ 53, 187,  92],
        ...,
        [210, 225,  70],
        [ 70, 200, 110],
        [141, 176, 177]],

       ...,

       [[  9,  58, 102],
        [112, 131, 190],
        [  8, 223,  40],
        ...,
        [167, 135, 124],
        [  6,  28,   5],
        [248, 237, 115]],

       [[211, 216, 254],
        [  1,  97, 203],
        [221, 253, 196],
        ...,
        [244, 191, 253],
        [150, 163, 170],
        [ 79,  42, 153]],

       [[ 54,  66,  25],
        [230, 151, 224],
        [  0, 219,  77],
        ...,
        [244,   1,  90],
        [  5, 211, 172],
        [173,   5,  36]]

# 3. Train Model

In [4]:
log_path = os.path.join('Training', 'Logs')


In [5]:
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=log_path)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


c:\Users\rahul\OneDrive\Desktop\Synaptic_RL\.venv\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [7]:
# Wrapper to ensure actions are native Python floats
class ActionFloatWrapper(gym.ActionWrapper):
	def action(self, action):
		return action.astype(float)

# Re-wrap the environment if not already wrapped
env = ActionFloatWrapper(env)

model.set_env(env)
model.learn(total_timesteps=40000)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to Training\Logs\PPO_9


c:\Users\rahul\OneDrive\Desktop\Synaptic_RL\.venv\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -56.1    |
| time/              |          |
|    fps             | 40       |
|    iterations      | 1        |
|    time_elapsed    | 50       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -51.6       |
| time/                   |             |
|    fps                  | 34          |
|    iterations           | 2           |
|    time_elapsed         | 119         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005605802 |
|    clip_fraction        | 0.0549      |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.25       |
|    explained_variance   | 0.00443     |
|    learning_rate        | 0.

# 4. Save Model 

In [8]:
ppo_path = os.path.join('Training', 'Saved Models', 'PPO_Driving_model')


In [9]:
model.save(ppo_path)


# 5. Evaluate and Test

In [10]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)


c:\Users\rahul\OneDrive\Desktop\Synaptic_RL\.venv\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(np.float64(-50.72966934144497), np.float64(5.549721147919671))

In [11]:
env.close()


In [22]:
env = ActionFloatWrapper(env)


In [25]:
obs, info = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, terminated, truncated, info = env.step(action)
    env.render()
    if terminated or truncated:
        break


In [26]:
env.close()
